In [1]:
from datetime import datetime
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
from geopy.distance import geodesic

# %matplotlib widget
%matplotlib inline

In [2]:
bs = gpd.read_file("../processed_gbfs_data/bikeshare_2025-02-28_15-44-09.geojson")

In [3]:
my_date = "2025-02-22"
my_date = datetime.strptime(my_date, "%Y-%m-%d")
my_date, my_date + pd.Timedelta(days=1)
my_date_mask = (bs.datetime > my_date) & (bs.datetime < my_date + pd.Timedelta(days=1))

In [22]:
bs["bike_id"].unique().shape

(2497160,)

In [29]:
bs.last_reported

0          1740410972
1          1740410972
2          1740410972
3          1740410972
4          1740410972
              ...    
2535898    1740531577
2535899    1740531577
2535900    1740531577
2535901    1740531577
2535902    1740531577
Name: last_reported, Length: 2535903, dtype: int32

In [38]:
bs.groupby("bike_id").agg("count")["lat"].value_counts()

lat
1    2458417
2      38743
Name: count, dtype: int64

In [35]:
bs.shape

(2535903, 11)

In [42]:
(2458417 / (2458417 + 38743)), (38743 / (2458417 + 38743))

(0.9844851751589806, 0.015514824841019398)

In [24]:
bs[my_date_mask].shape

(378187, 11)

In [4]:
def calculate_distance(df):
    df = df.sort_values(by="datetime")  # Ensure time order
    df['prev_lat'] = df['lat'].shift(1)
    df['prev_lon'] = df['lon'].shift(1)
    # Calculate distance in meters
    df['distance_m'] = df.apply(
        lambda row: geodesic((row['prev_lat'], row['prev_lon']), (row['lat'], row['lon'])).meters
        if pd.notnull(row['prev_lat']) else 0, axis=1)
    return df

In [10]:
calculate_distance(bs[my_date_mask])

,bike_id,lat,lon,is_reserved,is_disabled,current_range_meters,vehicle_type_id,last_reported,vehicle_type,datetime,geometry,prev_lat,prev_lon,distance_m
1820613,25a4e1e5-f405-4def-a903-fc4eb42606e8,47.602039,-122.332124,False,False,13861,3,1740211218,e-bike,2025-02-22 00:00:31,POINT (550202.391 5272286.133),NaN,NaN,0.000000
1825647,58ef6c98-f6f3-43fa-b220-9f4b17cd731f,47.613210,-122.344042,False,False,36057,2,1740211221,scooter,2025-02-22 00:00:31,POINT (549296.053 5273520.008),47.602039,-122.332124,1531.544182
1825646,a36281e3-923e-4207-8445-89fd0a59f817,47.616543,-122.344026,False,False,19975,2,1740211221,scooter,2025-02-22 00:00:31,POINT (549294.123 5273890.44),47.613210,-122.344042,370.574313
1825645,163c5b88-1f04-46c8-ab1a-eb0298461bc8,47.615375,-122.343978,False,False,18693,3,1740211221,e-bike,2025-02-22 00:00:31,POINT (549298.828 5273760.661),47.616543,-122.344026,129.911692
1825644,0764bec7-d5b2-4b84-a018-8e2e03a1e0ee,47.615269,-122.343954,False,False,6556,2,1740211221,scooter,2025-02-22 00:00:31,POINT (549300.731 5273748.896),47.615375,-122.343978,11.922694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1758206,ea95b3ef-d7d3-46b9-9317-ddebcea152e2,47.608129,-122.330173,False,False,24467,2,1740295789,scooter,2025-02-22 23:30:19,POINT (550343.197 5272964.225),47.611795,-122.330236,407.623413
1758205,45168e42-1419-4c05-8915-b010893b32db,47.607999,-122.330004,False,False,10200,2,1740295789,scooter,2025-02-22 23:30:19,POINT (550356.024 5272949.886),47.608129,-122.330173,19.245169
1758204,ccf10012-d4e7-40d6-97a3-5fa15ffcb330,47.608152,-122.329863,False,False,38808,2,1740295789,scooter,2025-02-22 23:30:19,POINT (550366.474 5272966.982),47.607999,-122.330004,20.044124
1758217,910b7bb3-41e0-4e46-aa62-fff0c7084356,47.607080,-122.330662,False,False,3873,2,1740295789,scooter,2025-02-22 23:30:19,POINT (550307.451 5272847.324),47.608152,-122.329863,133.472518


In [5]:
bs[my_date_mask].head()

,bike_id,lat,lon,is_reserved,is_disabled,current_range_meters,vehicle_type_id,last_reported,vehicle_type,datetime,geometry
72111,348a28fe-951e-4339-8b05-6ab85f75dd3e,47.520074,-122.262508,False,False,4446,3,1740234626,e-bike,2025-02-22 06:30:52,POINT (555521.744 5263224.225)
72112,363142c1-79c3-47a7-bd36-82491ca90de3,47.523507,-122.263511,False,False,21732,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555442.617 5263605.038)
72113,6eed629e-a321-48f4-a020-4d2da084a342,47.538952,-122.264554,False,False,31826,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555347.847 5265320.788)
72114,fc860a02-cb1a-4952-be1d-510d64fa0084,47.538826,-122.264676,False,False,38251,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555338.799 5265306.698)
72115,b47f9849-4eae-4717-bb4a-905ee1b5eac9,47.525007,-122.264746,False,False,34979,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555348.069 5263770.861)


In [6]:
res = bs[my_date_mask].groupby("bike_id", group_keys=False).apply(calculate_distance)

/var/folders/5d/g9psjp7j7jl8l_mm5697j11c0000gn/T/ipykernel_35480/492732165.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  res = bs[my_date_mask].groupby("bike_id", group_keys=False).apply(calculate_distance)


In [8]:
bs[my_date_mask].head()

,bike_id,lat,lon,is_reserved,is_disabled,current_range_meters,vehicle_type_id,last_reported,vehicle_type,datetime,geometry
72111,348a28fe-951e-4339-8b05-6ab85f75dd3e,47.520074,-122.262508,False,False,4446,3,1740234626,e-bike,2025-02-22 06:30:52,POINT (555521.744 5263224.225)
72112,363142c1-79c3-47a7-bd36-82491ca90de3,47.523507,-122.263511,False,False,21732,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555442.617 5263605.038)
72113,6eed629e-a321-48f4-a020-4d2da084a342,47.538952,-122.264554,False,False,31826,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555347.847 5265320.788)
72114,fc860a02-cb1a-4952-be1d-510d64fa0084,47.538826,-122.264676,False,False,38251,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555338.799 5265306.698)
72115,b47f9849-4eae-4717-bb4a-905ee1b5eac9,47.525007,-122.264746,False,False,34979,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555348.069 5263770.861)


In [ ]:
bs[my_date_mask].groupby("bike_id", group_keys=False).apply()

In [7]:
res.head()

,bike_id,lat,lon,is_reserved,is_disabled,current_range_meters,vehicle_type_id,last_reported,vehicle_type,datetime,geometry,prev_lat,prev_lon,distance_m
72111,348a28fe-951e-4339-8b05-6ab85f75dd3e,47.520074,-122.262508,False,False,4446,3,1740234626,e-bike,2025-02-22 06:30:52,POINT (555521.744 5263224.225),NaN,NaN,0
72112,363142c1-79c3-47a7-bd36-82491ca90de3,47.523507,-122.263511,False,False,21732,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555442.617 5263605.038),NaN,NaN,0
72113,6eed629e-a321-48f4-a020-4d2da084a342,47.538952,-122.264554,False,False,31826,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555347.847 5265320.788),NaN,NaN,0
72114,fc860a02-cb1a-4952-be1d-510d64fa0084,47.538826,-122.264676,False,False,38251,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555338.799 5265306.698),NaN,NaN,0
72115,b47f9849-4eae-4717-bb4a-905ee1b5eac9,47.525007,-122.264746,False,False,34979,2,1740234626,scooter,2025-02-22 06:30:52,POINT (555348.069 5263770.861),NaN,NaN,0


0.9149999999999999